# ThermoCodegen integration with ThermoEngine

This notebook is designed to test the ability to include a ThermoCodegen generated thermodynamic database as a ThermoEngine model.Database object

In [ ]:
import numpy as np
from glob import glob
import os
import pytest

from thermoengine import model
from thermocodegen.testing import Tester
import thermocodegen.testing as tt

#### set up some directories

In [ ]:
try:
    os.makedirs('tests')
except:
    pass

try:
    os.makedirs('csvs')
except:
    pass

#### import the ThermoCodegen object

In [ ]:
import py_fo_sio2_poly_linear_rxns as pfr
pfr.phase_info()


In [ ]:
rxns = pfr.fo_sio2_poly_linear_rxns()
rxns.report()

In [ ]:
model_tcg=model.Database.from_thermocodegen(pfr)

In [ ]:
model_tcg.phase_info

##  Let's get serious...I want to compare tests between the pure tcg object and its wrapped thermoengine object

* `t_tcg` is a tester class for the direct ThermoCodegen pybind11 object
*  `t_te` is a testerclass for its wrapping as a ThermoEngine model.Database 
* `t_tedb` is a tester for the default ThermoEngine model.DataBase()

In [ ]:
t_tcg = Tester.factory(pfr)
t_te = Tester.factory(model_tcg)
t_tedb = Tester.factory(model.Database())
print(type(t_tcg))
print(type(t_te))
print(type(t_tedb))

##  List endmembers

These have different definitions between ThermoCodegen objects and ThermoEngine.
* For TCG:  all phases are built from endmembers, and pure phases simply have a single endmember
* For TE:  endmembers are all pure phases such that G(T,P)

so for ThermoCodegen,  all thermodynamic endmembers (including in solution phases and any other endmember in the database are exposed)

In [ ]:
t_tcg.endmembers

When wrapped as a thermoengine model, however,  only the pure-phases get returned as endmembers, and only the abbrevs are returned in the list

In [ ]:
t_te.endmembers

This is consistent with endmembers for the standard berman data base pure phases

In [ ]:
print(t_tedb.endmembers)

##  List Phases

Phases also have different definitions between ThermoCodegen objects and ThermoEngine.
* For TCG:  the phase dictionary includes both pure and solution phases (distinguished only by the number of endmembers)
* For TE:  phases are only solution phases

In [ ]:
t_tcg.phase_dict

In [ ]:
t_te.phase_dict

In [ ]:
t_tedb.phase_dict

#  Test Endmembers/pure phases

Need to figure out how to generate comprehensive tests for endmembers/pure phases

###  Test ThermoCodegen endmembers

get raw input dataframe

In [ ]:
dfe_in_tcg = t_tcg.get_endmember_df_in()
dfe_in_tcg

Set reference Temperature and pressure

In [ ]:
T=1700.
P=1000.

Restrict input dataframe to just endmember name, temperature and pressure and fill in

In [ ]:
dfe_in_tcg = dfe_in_tcg[['name', 'T', 'P']]

N = len(dfe_in_tcg)
dfe_in_tcg['T'] = T*np.ones(N)
dfe_in_tcg['P'] = P*np.ones(N)
dfe_in_tcg

Construct output dataframe from input dataframe

In [ ]:
dfe_out_tcg = t_tcg.set_df_out(dfe_in_tcg)
dfe_out_tcg

Output data frame as csv file and pytest object

In [ ]:
fileroot='tests/test_endmembers_tcg'
dfe_out_tcg.to_csv('csvs/Endmembers_tcg.csv')
t_tcg.write_tests(dfe_out_tcg, fileroot=fileroot)

Run pytest on the testfile

In [ ]:
testfile = glob(fileroot+'*')[0]
pytest.main(['--disable-warnings',testfile ])

## Repeat this for ThermoEngine Pure phases

Problem is that it can't distinguish function arguments particularly well and all pure phases have a bunch of default values of
mol=None and V=None so must include them in the input dataframe

get raw input dataframe

In [ ]:
dfe_in_te = t_te.get_endmember_df_in()
dfe_in_te

Restrict input dataframe to just endmember name, temperature, pressure and optional arguments mol and V and fill in

In [ ]:
dfe_in_te = dfe_in_te[['name', 'T', 'P', 'mol', 'V']]
N = len(dfe_in_te)
dfe_in_te['T'] = T*np.ones(N)
dfe_in_te['P'] = P*np.ones(N)
dfe_in_te['mol'] = [ None for n in range(N)]
dfe_in_te['V'] = [ None for n in range(N)]
dfe_in_te

Construct output dataframe (should work for all private _calc_? methods

In [ ]:
dfe_out_te = t_te.set_df_out(dfe_in_te)
dfe_out_te

Output data frame as csv file and pytest object

In [ ]:
fileroot='tests/test_endmembers_te'
dfe_out_tcg.to_csv('csvs/Endmembers_te.csv')
t_te.write_tests(dfe_out_te, fileroot=fileroot)

run pytest

In [ ]:
testfile = glob(fileroot+'*')[0]
pytest.main(['--disable-warnings',testfile ])

#  Test Solution phases

Again,  there is a difference in definition between ThermoCodegen and ThermoEngine on what constitutes a Phase.  For ThermoEngine a Phase is always a solution phase (so in this case we only need to test for the liquid phase

###  Test ThermoCodegen solution phase

get raw input dataframe

In [ ]:
dfp_in_tcg = t_tcg.get_phase_df_in()
dfp_in_tcg = dfp_in_tcg[dfp_in_tcg['name']=='Liquid']
dfp_in_tcg

Get Liquid phase object

In [ ]:
Lq = pfr.Liquid()

Set reference Temperature pressure and composition

In [ ]:
T=1700.
P=1000.
# endmembers are Si204 and Mg2SiO4
c = [.2, .8]
n = Lq.c_to_x(c)
x = n
print(c,n,x)

Restrict input dataframe to just endmember name, temperature and pressure and compositional variables

In [ ]:
dfp_in_tcg = dfp_in_tcg[['name', 'T', 'P', 'c', 'n', 'x']]

N = len(dfp_in_tcg)
dfp_in_tcg['T'] = T*np.ones(N)
dfp_in_tcg['P'] = P*np.ones(N)
dfp_in_tcg['c'] = [c]
dfp_in_tcg['n'] = [n]
dfp_in_tcg['x'] = [x]
dfp_in_tcg

Construct output dataframe

In [ ]:
dfp_out_tcg = t_tcg.set_df_out(dfp_in_tcg)
dfp_out_tcg

Output data frame as csv file and pytest object

In [ ]:
fileroot='tests/test_phases_tcg'
dfp_out_tcg.to_csv('csvs/Phases_tcg.csv')
t_tcg.write_tests(dfp_out_tcg, fileroot=fileroot)

run pytest

In [ ]:
testfile = glob(fileroot+'*')[0]
pytest.main(['--disable-warnings',testfile ])

###  Test ThermoEngine Wrapped phases

get raw input dataframe

In [ ]:
dfp_in_te = t_te.get_phase_df_in()
dfp_in_te

Restrict input dataframe to just phase name, temperature and pressure, moles and V

In [ ]:
dfp_in_te = dfp_in_te[['name', 'T', 'P', 'mol', 'V']]

N = len(dfp_in_te)
dfp_in_te['T'] = T*np.ones(N)
dfp_in_te['P'] = P*np.ones(N)
dfp_in_te['mol'] = [n]
dfp_in_te['V'] = None
dfp_in_te

set output data frame

In [ ]:
dfp_out_te = t_te.set_df_out(dfp_in_te)
dfp_out_te

Output data frame as csv file and pytest object

In [ ]:
fileroot='tests/test_phases_te'
dfp_out_tcg.to_csv('csvs/Phases_te.csv')
t_te.write_tests(dfp_out_te, fileroot=fileroot)

run pytest

In [ ]:
testfile = glob(fileroot+'*')[0]
pytest.main(['--disable-warnings',testfile ])